In [ ]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
from osgeo import osr, ogr


In [ ]:
def isin(string_,list_):
    return np.any([x in string_ for x in list_])

## Read data from spreadsheet

In [ ]:
key = '1-l2cmYpxRIplA8fRVm2qeZ52HeB2KNVc0gPnD2QHjmk'
sheet_name = 'Tiles to label'
string_ = f'https://docs.google.com/spreadsheet/ccc?key={key}&output=csv&sheet={sheet_name}'

response = requests.get(string_)
assert response.status_code == 200, 'Wrong status code'

In [ ]:
print(string_)
# response.content

In [ ]:

df = pd.read_csv(StringIO(response.content.decode('utf-8')), sep=",", skiprows=1, usecols=range(9),index_col=2,
                dtype={'Has Priority':str,
                      'Responsible':str})
df = df[df.index.notnull()]

df['Has Priority'].fillna('', inplace=True)
df['Responsible'].fillna('', inplace=True)
def define_labeled(x):
    if np.isnan(x['Areas Labeled']):
        return 'No' if x['Responsible'] == '' else 'inprogress'
    else:
        return 'Yes'
        
df['labeled'] = df.apply(define_labeled, axis=1)


In [ ]:
df['GT'] = df.apply(lambda x: x['Has Priority']+'_'+x['labeled'], axis=1)

In [ ]:
dict_update = df['GT'].to_dict()

In [ ]:
df[df.index == '47NRG']

## Regionsate Sentinel-2 features with GT status

In [ ]:
shp_file = '/scratch2/Dropbox/Dropbox/0_phd/yield/labels/sentinel2tiles/features_lab/Features.shp'


In [ ]:
input_zone_polygon = shp_file
is_update= True
fieldname='labeled'
# test_update = {'46NGL':'Yes'}
# input_value_raster = , fieldname, fn, fieldname1=None, is_update=True):

In [ ]:
# def loop_zonal_stats_update(input_zone_polygon, input_value_raster, fieldname, fn, fieldname1=None, is_update=True):

shp = ogr.Open(input_zone_polygon, update=1)
lyr = shp.GetLayer()
lyrdf =lyr.GetLayerDefn()

# TreeFieldName = 'TreePredAd1'
if is_update:
    id_ = lyrdf.GetFieldIndex(fieldname)
    if id_ == -1:
        field_defn = ogr.FieldDefn(fieldname, ogr.OFTString)
        lyr.CreateField(field_defn)
        id_ = lyrdf.GetFieldIndex(fieldname)
    else:
        print('Field {} already exists, may overwrite'.format(fieldname))

#     if fieldname1 is not None:
#         id_1 = lyrdf.GetFieldIndex(fieldname1)
#         if id_1 == -1:
#             field_defn = ogr.FieldDefn(fieldname1, ogr.OFTString)
#             lyr.CreateField(field_defn)
#             id_1 = lyrdf.GetFieldIndex(fieldname1)
#         else:
#             print('Field {} already exists, may overwrite'.format(fieldname1))
#     else:
#         id_1 = None


id_Name = lyrdf.GetFieldIndex('Name')
for FID in range(lyr.GetFeatureCount()):
    feat = lyr.GetFeature(FID)
    if feat is not None:
        name_ = feat.GetField(id_Name)
        if name_ in dict_update.keys():
            val = dict_update[name_]
#             print(name_, val)
            
            if is_update:
    #             if id_1 is not None:
    #                 val, val1 = val
    #                 lyr.SetFeature(feat)
    #                 feat.SetField(id_1, val1)
    #                 lyr.SetFeature(feat)
                lyr.SetFeature(feat)
                feat.SetField(id_,val)
                lyr.SetFeature(feat)



## Val areas for palm4

In [ ]:
# regions_ = ['Kalimantan Barat', 'Sarawak','Sabah', 'Kalimantan Utara']
regions_ = ['Sulawesi', 'Gorontalo','Maluku','Papua','Irian Jaya Barat']
tiles = list(df.loc[df.apply(lambda x: isin(x.Regions,regions_) and x['Areas Labeled'] > 0, axis=1)].index)
tiles = [f'T{x}' for x in tiles]
print(len(tiles))
','.join(tiles)

In [ ]:
tiles = list(df.loc[df.apply(lambda x: 'Kalimantan Barat' in x.Regions, axis=1)].index)
tiles = ['T'+x for x in tiles]
print(len(tiles))
' '.join(tiles)

## All areas to predict in Indo and Malaysia

In [ ]:
df

In [ ]:
# regions_ = ['Kalimantan Barat', 'Sarawak','Sabah', 'Kalimantan Utara']
regions_ = ['Indonesia', 'Malaysia']
tiles = list(df.loc[df.apply(lambda x: isin(x.Country,regions_), axis=1)].index)
tiles = [f'T{x}' for x in tiles]
print(len(tiles))
' '.join(tiles)

In [ ]:
tiles = list(df.loc[df.apply(lambda x: 'Kalimantan Barat' in x.Regions, axis=1)].index)
tiles = ['T'+x for x in tiles]
print(len(tiles))
' '.join(tiles)

In [ ]:
#regions_ = ['Kalimantan Barat', 'Sarawak','Sabah', 'Kalimantan Utara']
regions_ = ['Kalimantan Tengah', 'Kalimantan Selatan','Kalimantan Timur']

tiles = list(df.loc[df.apply(lambda x: isin(x.Regions,regions_) and x['Areas Labeled'] > 0, axis=1)].index)
tiles = [f'T{x}' for x in tiles]
print(len(tiles))
','.join(tiles)

In [ ]:
#regions_ = ['Kalimantan Barat', 'Sarawak','Sabah', 'Kalimantan Utara']
regions_ = ['Riau']

tiles = list(df.loc[df.apply(lambda x: isin(x.Regions,regions_) and x['Areas Labeled'] > 0, axis=1)].index)
tiles = [f'T{x}' for x in tiles]
print(len(tiles))
','.join(tiles)

In [ ]:
# regions_ = ['Kalimantan Barat', 'Sarawak','Sabah', 'Kalimantan Utara']
regions_ = ['Kalimantan','Sarawak','Sabah']
tiles = list(df.loc[df.apply(lambda x: isin(x.Regions,regions_) and x['Areas Labeled'] > 0, axis=1)].index)
tiles = [f'T{x}' for x in tiles]
print(len(tiles))
','.join(tiles)

In [ ]:
# regions_ = ['Kalimantan Barat', 'Sarawak','Sabah', 'Kalimantan Utara']
regions_ = ['Sulawesi', 'Gorontalo']
tiles = list(df.loc[df.apply(lambda x: (isin(x.Regions,regions_) or x.Country == 'Phillipines') and x['Areas Labeled'] > 0, axis=1)].index)
tiles = [f'T{x}' for x in tiles]
print(len(tiles))
','.join(tiles)

In [ ]:
#regions_ = ['Kalimantan Barat', 'Sarawak','Sabah', 'Kalimantan Utara']
regions_ = ['Kalimantan', 'Sarawak', 'Sabah']

tiles = list(df.loc[df.apply(lambda x: isin(x.Regions,regions_), axis=1)].index)
tiles = [f'T{x}' for x in tiles]
print(len(tiles))
' '.join(tiles)

In [ ]:
#regions_ = ['Kalimantan Barat', 'Sarawak','Sabah', 'Kalimantan Utara']
regions_ = ['Sulawesi', 'Gorontalo']

tiles = list(df.loc[df.apply(lambda x: isin(x.Regions,regions_), axis=1)].index)
tiles = [f'T{x}' for x in tiles]
print(len(tiles))
' '.join(tiles)

In [ ]:
#regions_ = ['Kalimantan Barat', 'Sarawak','Sabah', 'Kalimantan Utara']
regions_ = ['Palawan']

tiles = list(df.loc[df.apply(lambda x: isin(x.Regions,regions_), axis=1)].index)
tiles = [f'T{x}' for x in tiles]
print(len(tiles))
' '.join(tiles)

In [ ]:
tiles = set(df.loc[np.logical_and(df.Country == 'Phillipines'].index)
tiles = [f'T{x}' for x in tiles]
print(len(tiles))
' '.join(tiles)

In [ ]:
tiles = set(df.loc[df.Country == 'Phillipines'].index)
tiles = sorted([f'T{x}' for x in tiles])
print(len(tiles))
' '.join(tiles)

In [ ]:
isin(df.index,['46','47'])

In [ ]:
# regions_ = ['Kalimantan Barat', 'Sarawak','Sabah', 'Kalimantan Utara']
regions_ = ['47','48']
tiles = list(df.loc[df.apply(lambda x: isin(x.index,regions_) and x['Areas Labeled'] > 0, axis=1)].index)
tiles = [f'T{x}' for x in tiles]
print(len(tiles))
','.join(tiles)

In [ ]:
# regions_ = ['Kalimantan Barat', 'Sarawak','Sabah', 'Kalimantan Utara']
regions_ = ['47','48']
tiles = list(df[[isin(x,regions_) for x in df.index]].index)
tiles = [f'T{x}' for x in tiles]
print(len(tiles))
' '.join(tiles)

In [ ]:
df[[isin(x,regions_) for x in df.index]].index